<a href="https://colab.research.google.com/github/arunm917/CS6910_Assignment1/blob/main/CS6910_Assignment_1_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing packages

In [2]:
import numpy as np
import pandas as pd
import gdown
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm.notebook import trange, tqdm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
# import tensorflow as ts
from tensorflow import keras
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Data preprocessing - fashion MNIST

In [3]:
(X, y), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, random_state=7, test_size = 0.2)

In [6]:
X_train_scaled = X_train.reshape(len(X_train),28*28)/255.0
X_val_scaled = X_val.reshape(len(X_val),28*28)/255.0
X_test_scaled = X_test.reshape(len(X_test),28*28)/255.0

Encoding labels

In [38]:
enc = OneHotEncoder()
y_train_enc = enc.fit_transform(np.expand_dims(y_train,1)).toarray()
y_val_enc = enc.fit_transform(np.expand_dims(y_val,1)).toarray()
y_test_enc = enc.fit_transform(np.expand_dims(y_test,1)).toarray()
print(y_train_enc.shape, y_val_enc.shape, y_test_enc.shape)

(48000, 10) (12000, 10) (10000, 10)


Batching

In [8]:
X_train_batch = X_train_scaled[0:1000,:]
y_train_batch = y_train_enc[0:1000,:]

In [9]:
print(X_train_scaled.shape, y_train_enc.shape, X_train_batch.shape, y_train_batch.shape )

(48000, 784) (48000, 10) (1000, 784) (1000, 10)


# Initializations

In [11]:
class Initialization:
  
  def initialize(self, sizes):
    self.W = {}
    self.B = {}
    self.sizes = sizes
    self.nh = len(sizes)-2

    for i in range(self.nh + 1):
      self.W[i+1] = np.random.randn(self.sizes[i+1], self.sizes[i])
      self.B[i+1] = np.zeros((self.sizes[i+1],1))

    return(self.W, self.B)
    
initial = Initialization()  #Instantiation for the class initialization

# Hyperparameters

In [42]:
class Hyperparameters:
  epochs = 20
  eta    = 1e-5

hyper = Hyperparameters()

# Activation functions

In [63]:
class ActivationFunctions:

  def linear(self,W,b,x):
    return (np.dot(W,x) + b)

  def sigmoid(self, z):
    return 1/(1+np.exp(-z))

  def softmax(self, z):
    return np.exp(z)/np.sum(np.exp(z))
  
  def grad_sigmoid(self,z):
    return z*(1-z)


act = ActivationFunctions()

# Loss Functions

In [61]:
class LossFunctions:
  def MSE(self, y, y_pred):
    return (np.sum((y-y_pred)**2))/len(y)
  
  def cross_entropy(self, y, y_pred):
    ce_loss = np.mean(np.sum(np.multiply(-y,np.log(y_pred)), axis = 0))
    #ce_loss = log_loss(y,y_pred)
    #print(log_loss(y,y_pred))
    return(ce_loss)
loss_functions = LossFunctions()

# Learning Algorithms

In [14]:
class Optimizer:

  def learning_algoithm(self, W, B, dW, dB, nh, learning_rate, optimizer):
    if optimizer == 'SGD':
      for i in range(nh + 1):
        W[i+1] -= learning_rate * (dW[i+1])
        B[i+1] -= learning_rate * (dB[i+1])
  
    return(W, B)

opt = Optimizer()

# Model

In [62]:
class Model(Optimizer):
    
  def forward_prop(self, x,y):
    self.A = {}
    self.H = {}
    self.x = x
    self.y = y
    loss_batch = 0
    self.H[0] = self.x.T

    for i in range(self.nh + 1):
      self.A[i+1] = np.matmul(self.W[i+1], self.H[i]) + self.B[i+1]
      self.H[i+1] = act.sigmoid(self.A[i+1])

    y_pred = act.softmax(self.A[i+1])
    #print(y_pred)
    #print('Y = ', self.y.shape, 'Y_pred =', y_pred.shape)
    loss_batch = loss_functions.cross_entropy(self.y.T ,y_pred)
    self.loss_batch_store.append(loss_batch)
    return(y_pred)
  
  def grad(self, x, y):

    y_pred = self.forward_prop(x,y)
    self.dW = {}
    self.dB = {}
    self.dA = {}
    self.dH = {}
    self.dA[self.nh + 1] = y_pred - y.T

    for i in range(self.nh + 1 , 0, -1):
      self.dW[i]      = np.matmul(self.dA[i],self.H[i-1].T)
      self.dB[i]      = np.sum(self.dA[i], axis = 1).reshape(-1,1)
      self.dH[i-1]    = np.matmul(self.W[i].T,self.dA[i])
      self.dA[i-1]    = np.multiply(self.dH[i-1],act.grad_sigmoid(self.H[i-1]))
    return (self.dW, self.dB)
  
  def fit(self, X, Y, epochs, learning_rate, hidden_layer_sizes, batch_size, optimizer):
    
    # x = X[0:batch_size,:]
    # y = Y[0:batch_size,:]
    #self.X = X
    self.loss_epoc_store = []
    self.nx = X.shape[1]
    self.ny = Y.shape[1]
    self.nh = len(hidden_layer_sizes)
    self.sizes = [self.nx] + hidden_layer_sizes + [self.ny]
    step = len(X)/batch_size
    #print(len(X))
    #print('step size =',step)
    self.W, self.B = initial.initialize(self.sizes)

    for i in trange(epochs, total=epochs, unit="epoch"):
      loss_epoc = 0
      self.loss_batch_store = []
      for i in range(0,len(X),batch_size):
        self.x = X[i:i+batch_size,:]
        self.y = Y[i:i+batch_size,:]
        #print('y =', y)
        self.grad(self.x,self.y)
        self.W, self.B = opt.learning_algoithm(self.W, self.B, self.dW, self.dB, self.nh, learning_rate, optimizer)

      loss_epoc = np.sum(self.loss_batch_store)/step
      print('training loss = ', loss_epoc)
      self.loss_epoc_store.append(loss_epoc)
    
    plt.plot(self.loss_epoc_store)
    plt.xlabel('Epochs')
    plt.ylabel('log_loss')
    plt.show()

model = Model()

In [ ]:
model.fit(X_train_scaled, y_train_enc, 100, 1e-5, [512, 256], 25, optimizer = 'SGD')

In [56]:
len(X_train_batch)

1000

In [ ]:
batch_size = 25
for i in range(0,len(X_train_batch),batch_size):
  x = X_train_batch[i:i+batch_size,:]
  y = y_train_batch[i:i+batch_size,:]
  print(i)

In [91]:
x.shape

(25, 784)

In [ ]:
class Train:

  

    #     y_pred = self.forward_prop(x, hidden_layer_sizes)
    #     loss = log_loss(y,y_pred)
    #     loss_gd.append(loss)

    #     for i in range(self.nh + 1):
    #       self.W[i+1] -= learning_rate * (self.dW_sum[i+1]/m)
    #       self.B[i+1] -= learning_rate * (self.dB_sum[i+1]/m)
          
    #   loss_per_epoc = (np.sum(loss_gd))/m
    #   self.loss_per_epoc_store.append(loss_per_epoc)
    #   print('training loss = ', loss_per_epoc, 'learning rate = ', learning_rate)
    
    # plt.plot(self.loss_per_epoc_store)
    # plt.xlabel('Epochs')
    # plt.ylabel('log_loss')
    # plt.show()

train = Train()

In [ ]:
train.fit(X_train_scaled, y_train_enc, 10, 1e-3, [512, 256], 25, 'sgd')

x (25, 784) y (25, 10)


  0%|          | 0/10 [00:00<?, ?epoch/s]

forward prop
nx = 784
nh = 2
sizes = [784, 512, 256, 10]
W = 1 (512, 784)
B = 1 (512, 1)
W = 2 (256, 512)
B = 2 (256, 1)
W = 3 (10, 256)
B = 3 (10, 1)
0
A = 1 (512, 25)
H = 1 (512, 25)
1
A = 2 (256, 25)
H = 2 (256, 25)
2
A = 3 (10, 25)
H = 3 (10, 25)
y_pred = (10, 25)
end of forward_prop
dA 3 (10, 25)
dW 3 (10, 256)
dB 3 (10, 1)
dH 2 (256, 25)
dA 2 (256, 25)
dW 2 (256, 512)
dB 2 (256, 1)
dH 1 (512, 25)
dA 1 (512, 25)
dW 1 (512, 784)
dB 1 (512, 1)
dH 0 (784, 25)
dA 0 (784, 25)
forward prop
nx = 784
nh = 2
sizes = [784, 512, 256, 10]
W = 1 (512, 784)
B = 1 (512, 1)
W = 2 (256, 512)
B = 2 (256, 1)
W = 3 (10, 256)
B = 3 (10, 1)
0
A = 1 (512, 25)
H = 1 (512, 25)
1
A = 2 (256, 25)
H = 2 (256, 25)
2
A = 3 (10, 25)
H = 3 (10, 25)
y_pred = (10, 25)
end of forward_prop
dA 3 (10, 25)
dW 3 (10, 256)
dB 3 (10, 1)
dH 2 (256, 25)
dA 2 (256, 25)
dW 2 (256, 512)
dB 2 (256, 1)
dH 1 (512, 25)
dA 1 (512, 25)
dW 1 (512, 784)
dB 1 (512, 1)
dH 0 (784, 25)
dA 0 (784, 25)
forward prop
nx = 784
nh = 2
sizes = [